In [ ]:
DATA_DIR = "/opt/gpudata/skin-cancer/data"
OUTPUT_DIR = "/opt/gpudata/skin-cancer/outputs"

In [ ]:
# load cross validation folds
import json
import os

with open(os.path.join(DATA_DIR, "folds.json"), "r") as f:
    folds = json.load(f)

In [ ]:
# load slide embeddings
import pickle

with open(
    os.path.join(OUTPUT_DIR, "gigapath_slide_embeds_pool.pkl"), "rb"
) as f:
    embeds = pickle.load(f)

In [ ]:
# map specimen id to a list of WSIs
specs = {}
for slide in embeds.keys():
    key = slide[:6]
    if specs.get(key) is not None:
        specs[key].append(slide)
    else:
        specs[key] = [slide]

In [ ]:
# TODO: add "fold" col to dataframe and groupby to get grouped data instead
slide_folds = [[] for _ in range(5)]
for i, fold in enumerate(folds):
    for spec in fold:
        slide_folds[i].extend(specs[spec])

In [ ]:
# load data labels
import pandas as pd

labels = pd.read_csv(os.path.join(DATA_DIR, "labels/labels.csv"))
labels["specimenid"] = labels["specimenid"].astype("string")

In [ ]:
# create dataframe with slide data
import numpy as np

slide_ids = list(embeds.keys())
specimen_ids = [slide_id[:6] for slide_id in slide_ids]
embeddings = [embeds[slide_id] for slide_id in slide_ids]
df = pd.DataFrame(
    {
        "slide_id": slide_ids,
        "specimen_id": specimen_ids,
        "embedding": embeddings,
    }
)
df["slide_id"] = df["slide_id"].astype("string")
df["specimen_id"] = df["specimen_id"].astype("string")
df["embedding"] = df["embedding"].apply(lambda x: np.array(x.squeeze(0)))

In [ ]:
# merge slide data dataframe with labels dataframe
df = df.merge(
    labels[["specimenid", "bowens1", "scc1", "bcc1"]],
    how="right",
    left_on="specimen_id",
    right_on="specimenid",
)
df = df.set_index("slide_id")

In [ ]:
from enum import IntEnum


class Label(IntEnum):
    na = 0
    bowens = 1
    bcc = 2
    scc = 3


def get_labels(x):
    if x["bowens1"].item() == 1:
        return Label.bowens.value
    elif x["bcc1"].item() == 1:
        return Label.bcc.value
    elif x["scc1"].item() == 1:
        return Label.scc.value
    else:
        return Label.na.value

In [ ]:
X = df["embedding"]
y = df[["bowens1", "scc1", "bcc1"]]
y = y.apply(get_labels, axis=1)

In [ ]:
def train_val_split(
    X: pd.DataFrame, y: pd.DataFrame, val_fold_indices: list
) -> tuple:
    X_val = X.loc[val_fold_indices]
    X_train = X[X.index.difference(X_val.index)]

    y_val = y.loc[val_fold_indices]
    y_train = y[y.index.difference(y_val.index)]

    return (
        X_train.to_list(),
        y_train.to_list(),
        X_val.to_list(),
        y_val.to_list(),
    )

In [ ]:
def get_spec_level_probs(slide_indices: list, probs: list):
    combined_probs = {}
    for j, slide_idx in enumerate(slide_indices):
        spec = slide_idx[:6]
        if combined_probs.get(spec) is None:
            combined_probs[spec] = [probs[j]]
        else:
            combined_probs[spec].append(probs[j])

    for k in combined_probs.keys():
        combined_probs[k] = sum(combined_probs[k]) / len(combined_probs[k])

    return list(combined_probs.keys()), np.array(list(combined_probs.values()))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import PrecisionRecallDisplay, RocCurveDisplay, auc

%matplotlib inline


def plot_eval(
    mean_x: np.ndarray,
    onehot_labels: np.ndarray,
    probs: np.ndarray,
    ax: plt.Axes,
    type: str,
):
    # set eval type specific vars
    if type == "ROC":
        curve_display = RocCurveDisplay
        set_initial_interp_value = True
        interp_transform = lambda x: x
        x_attr = "fpr"
        y_attr = "tpr"
        auc_attr = "roc_auc"
    else:
        curve_display = PrecisionRecallDisplay
        set_initial_interp_value = False
        interp_transform = lambda x: np.flip(x)
        x_attr = "recall"
        y_attr = "precision"
        auc_attr = "average_precision"

    # create the curve
    # TODO: figure out chance level; should be a weighted average over
    #       folds for PRC
    curve = curve_display.from_predictions(
        onehot_labels,
        probs,
        name=f"{type} fold {i}",
        plot_chance_level=(i == len(slide_folds) - 1),
        ax=ax,
        alpha=0.3,
        lw=1,
    )

    # interpolate results using mean_x param to allow for averaging of curves
    interp_y = np.interp(
        mean_x,
        interp_transform(getattr(curve, x_attr)),
        interp_transform(getattr(curve, y_attr)),
    )
    interp_y[0] = 0.0 if set_initial_interp_value else interp_y[0]

    return interp_y, getattr(curve, auc_attr)


def create_mean_curve(
    mean_x: np.ndarray,
    cv_y: np.ndarray,
    aucs: np.ndarray,
    ax: plt.Axes,
    type: str,
    class_name: str,
):
    if type == "ROC":
        set_final_mean_value = True
        ax.set(
            xlabel="False Positive Rate",
            ylabel="True Positive Rate",
            title=f"Mean ROC curve with variability for {class_name}",
        )
    else:
        set_final_mean_value = False
        ax.set(
            xlabel="Recall",
            ylabel="Precision",
            title=f"Mean precision-recall curve with variability for {class_name}",
        )

    # prep plot data
    mean_y = np.mean(cv_y, axis=0)
    mean_y[-1] = 1.0 if set_final_mean_value else mean_y[-1]
    mean_auc = auc(mean_x, mean_y)
    std_auc = np.std(aucs)
    auc_label = "AUC" if type == "ROC" else "AP"

    # plot the mean line
    ax.plot(
        mean_x,
        mean_y,
        color="b",
        label=r"Mean %s (%s = %0.2f $\pm$ %0.2f)"
        % (type, auc_label, mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )

    # plot the standard deviation bands
    std_y = np.std(cv_y, axis=0)
    y_upper = np.minimum(mean_y + std_y, 1)
    y_lower = np.maximum(mean_y - std_y, 0)
    ax.fill_between(
        mean_x,
        y_lower,
        y_upper,
        color="grey",
        alpha=0.2,
        label=r"$\pm$ 1 std. dev.",
    )

    ax.legend(loc="lower right")

In [ ]:
def crossval(clf, folds: list, y_onehot: pd.DataFrame):
    # dictionaries to keep eval curve results for each label
    tprs = {label: [] for label in Label._member_names_}
    aucs = {label: [] for label in Label._member_names_}
    precisions = {label: [] for label in Label._member_names_}
    aps = {label: [] for label in Label._member_names_}

    # mean x vals and axes for curves
    mean_fpr = np.linspace(0, 1, 100)
    mean_recall = np.linspace(0, 1, 100)
    roc_fig, roc_axs = plt.subplots(2, 2, figsize=(12, 12))
    prc_fig, prc_axs = plt.subplots(2, 2, figsize=(12, 12))

    for val_fold_indices in folds:
        # fit the classifier on the train data and extract probs
        X_train, y_train, X_val, _ = train_val_split(X, y, val_fold_indices)
        clf.fit(X_train, y_train)
        probs = clf.predict_proba(X_val)

        # probs are on slide-level - need specimen level
        _, probs = get_spec_level_probs(val_fold_indices, probs)

        # get onehot labels for val set
        y_onehot_val = y_onehot.loc[val_fold_indices].drop_duplicates(
            subset=["specimen_id"]
        )

        # for each predicted class, plot the current classifier's eval
        # and retain relevant data in dicts
        for j, class_of_interest in enumerate(Label._member_names_):
            interp_tpr, roc_auc = plot_eval(
                mean_fpr,
                y_onehot_val[f"{class_of_interest}"],
                probs[:, Label[class_of_interest].value],
                roc_axs[j // 2][j % 2],
                "ROC",
            )
            tprs[class_of_interest].append(interp_tpr)
            aucs[class_of_interest].append(roc_auc)

            interp_precision, average_precision = plot_eval(
                mean_recall,
                y_onehot_val[f"{class_of_interest}"],
                probs[:, Label[class_of_interest].value],
                prc_axs[j // 2][j % 2],
                "PRC",
            )
            precisions[class_of_interest].append(interp_precision)
            aps[class_of_interest].append(average_precision)

    # plot the mean eval curves
    for j, class_of_interest in enumerate(Label._member_names_):
        create_mean_curve(
            mean_fpr,
            tprs[class_of_interest],
            aucs[class_of_interest],
            roc_axs[j // 2][j % 2],
            "ROC",
            class_of_interest,
        )
        create_mean_curve(
            mean_recall,
            precisions[class_of_interest],
            aps[class_of_interest],
            prc_axs[j // 2][j % 2],
            "PRC",
            class_of_interest,
        )

    roc_fig.show()
    prc_fig.show()

In [ ]:
y_onehot = df[["bowens1", "scc1", "bcc1", "specimen_id"]]
y_onehot["na"] = 1 - y_onehot.loc[:, ["bowens1", "scc1", "bcc1"]].sum(axis=1)
y_onehot = y_onehot.rename(
    columns={
        "bowens1": "bowens",
        "bcc1": "bcc",
        "scc1": "scc",
    }
)

In [ ]:
from xgboost import XGBClassifier

clf = XGBClassifier(objective="multi:softmax", num_class=4)
crossval(clf, slide_folds, y_onehot)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=500, solver="saga")
crossval(clf, slide_folds, y_onehot)